In [ ]:
from tensorflow.keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
import glob
import re
import random
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras.layers import Activation, MaxPool2D, Concatenate


In [ ]:
image_directory = "../input/dataset-deep-angiogram-png/dataset_deep_Angiogram_png/Original/train/"
mask_directory = "../input/dataset-deep-angiogram-png/dataset_deep_Angiogram_png/groundT/train/"

In [ ]:
SIZE = 256
num_images = 100

Load images and masks in order so they match

In [ ]:
image_dataset = []
images = os.listdir(image_directory)
images.sort(key=lambda f: int(re.sub('\D', '', f)))
for i, image_name in enumerate(images):
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(image_directory+image_name, 1)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))

image_dataset = np.array(image_dataset)/255

Read masks the same way.

In [ ]:
mask_dataset = []
masks = os.listdir(mask_directory)
masks.sort(key=lambda f: int(re.sub('\D', '', f)))
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))

mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.20, random_state = 42)

In [ ]:
image_number = random.randint(0, len(X_train)-1)
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(X_train[image_number,:,:,0], cmap='gray')
plt.subplot(122)
plt.imshow(y_train[image_number,:,:,0], cmap='gray')
plt.show()


# Building Unet by dividing encoder and decoder into blocks

In [ ]:
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

### Encoder block: Conv block followed by maxpooling

In [ ]:
def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    x = conv_block(x, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

###Decoder block
###skip features gets input from encoder for concatenation

In [ ]:
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

### Build Unet using the blocks

In [ ]:
def build_unet(input_shape, n_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 32)
    s2, p2 = encoder_block(p1, 64)
    s3, p3 = encoder_block(p2, 128)

    b1 = conv_block(p3, 256)

    d1 = decoder_block(b1, s3, 128)
    d2 = decoder_block(d1, s2, 64)
    d3 = decoder_block(d2, s1, 32)

    if n_classes == 1:
      activation = 'sigmoid'
    else:
      activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding="same", activation=activation)(d3)
    print(activation)

    model = Model(inputs, outputs, name="U-Net")
    return model

In [ ]:
IMG_HEIGHT = image_dataset.shape[1]
IMG_WIDTH  = image_dataset.shape[2]
IMG_CHANNELS = image_dataset.shape[3]

input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

In [ ]:
import tensorflow as tf
def dice(y_true, y_pred):

  y_true = tf.cast(y_true, tf.float32)
  numerator = 2 * tf.reduce_sum(y_true * y_pred)
  denominator = tf.reduce_sum(y_true + y_pred)

  return numerator / denominator

In [ ]:
def jaccard(y_true, y_pred):

  intersection = tf.reduce_sum(y_true * y_pred)
  sum_ = tf.reduce_sum(y_true + y_pred)
  jac = (intersection) / (sum_ - intersection)

  return jac

In [ ]:
from sklearn.metrics import jaccard_score, f1_score, confusion_matrix, \
                            roc_curve, roc_auc_score

In [ ]:
import keras

In [ ]:
model = build_unet(input_shape, n_classes=1)
model.compile(optimizer=Adam(learning_rate = 1e-3), \
              loss='binary_crossentropy', \
              metrics=['accuracy', jaccard, dice]
            )
model.summary()

In [ ]:
from datetime import datetime
start2 = datetime.now()
history = model.fit(X_train, y_train,
                    batch_size = 16,
                    verbose=1,
                    epochs=300,
                    validation_data=(X_test, y_test),
                    shuffle=False)
stop2 = datetime.now()

execution_time_Unet = stop2-start2
print("Linknet execution time is: ", execution_time_Unet)

In [ ]:
plt.figure(figsize=(10, 40))

for idx, key in enumerate(history.history.keys()):

    ax = plt.subplot(8, 2, 1 + idx)
    plt.title(key)
    plt.plot(history.history[key])

plt.show()

In [ ]:
model.save('model_Unet_300epochs.hdf5')

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
#Load previously saved model
from keras.models import load_model
model = load_model("/kaggle/working/model_Unet_300epochs.hdf5", compile=False)
#model = load_model("../input/model-unet2-150epochs/model_Unet2_150epochs.hdf5", compile=False)
print("Loaded...")

Loaded...


In [ ]:
y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.7

In [ ]:
from tensorflow.keras.metrics import MeanIoU

In [ ]:
n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)
IOU_keras.update_state(y_pred_thresholded, y_test)
print("Mean IoU =", IOU_keras.result().numpy())

Mean IoU = 0.7616256


In [ ]:
import statistics

L_dices=[]
for i in range(len(y_test)):

    score_dice=dice(y_test[i],y_pred[i])
    L_dices.append(score_dice.numpy())

print("Mean Dice =", statistics.mean(L_dices))

Mean Dice = 0.7209519


In [ ]:
import random

In [ ]:
threshold = 0.7
test_img_number = random.randint(0, len(X_test)-1)
test_img = X_test[test_img_number]
ground_truth=y_test[test_img_number]
test_img_input=np.expand_dims(test_img, 0)
print(test_img_input.shape)
prediction = (model.predict(test_img_input)[0,:,:,0] > 0.5).astype(np.uint8)
print(prediction.shape)

plt.figure(figsize=(16, 8))
plt.subplot(231)
plt.title('Testing Image')
plt.imshow(test_img[:,:,0], cmap='gray')
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth[:,:,0], cmap='gray')
plt.subplot(233)
plt.title('Prediction on test image')
plt.imshow(prediction, cmap='gray')

plt.show()